In [1]:
from keys import *

got OPENAI_API_KEY
got PINECONE_API_KEY
got PINECONE_ENV
got HUGGINGFACE_API_KEY
got SLACK_TOKEN
got SLACK_SIGNING_SECRET


In [ ]:
from pathlib import Path
from urllib.parse import urlparse

import pinecone
import requests
from langchain import LLMChain, PromptTemplate
from langchain.chains import (ConversationalRetrievalChain,
                              SimpleSequentialChain)
from langchain.document_loaders import (PagedPDFSplitter, TextLoader,
                                        UnstructuredHTMLLoader,
                                        UnstructuredMarkdownLoader,
                                        UnstructuredPDFLoader)
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import HuggingFacePipeline, OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

from keys import HUGGINGFACE_HEADERS
from slackbot import chain

# https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/pinecone.html?highlight=pinecone

In [ ]:
SENTIMENT_API_URL = "https://api-inference.huggingface.co/models/cardiffnlp/twitter-roberta-base-sentiment"
LANGUAGE_API_URL = "https://api-inference.huggingface.co/models/papluca/xlm-roberta-base-language-detection"
TOXIC_API_URL = "https://api-inference.huggingface.co/models/martin-ha/toxic-comment-model"
GIBBERISH_API_URL = "https://api-inference.huggingface.co/models/madhurjindal/autonlp-Gibberish-Detector-492513457"
Q_OR_S_API_URL = "https://api-inference.huggingface.co/models/shahrukhx01/question-vs-statement-classifier"
EMOTION_API_URL = "https://api-inference.huggingface.co/models/joeddav/distilbert-base-uncased-go-emotions-student"
OFFENSIVE_API_URL = "https://api-inference.huggingface.co/models/cardiffnlp/twitter-roberta-base-offensive"
GPT2_DETECTOR_API_URL = "https://api-inference.huggingface.co/models/roberta-large-openai-detector"

In [ ]:
def query(payload, url):
    response = requests.post(url, headers=HUGGINGFACE_HEADERS, json=payload)
    return response.json()
	
# These queries via huggingface are free but may need to wait for models to load.

sentiment_query = lambda p: query(p, SENTIMENT_API_URL)
language_query = lambda p: query(p, LANGUAGE_API_URL)
toxic_query = lambda p: query(p, TOXIC_API_URL)
gibberish_query = lambda p: query(p, GIBBERISH_API_URL)
q_or_s_query = lambda p: query(p, Q_OR_S_API_URL)
emotion_query = lambda p: query(p, EMOTION_API_URL)
offsensive_query = lambda p: query(p, OFFENSIVE_API_URL)
gpt2_detector_query = lambda p: query(p, GPT2_DETECTOR_API_URL)

#output = sentiment_query({
#	"inputs": "I like you. I love you",
#})

	
#language_query({
#	"inputs": "I like you. I love you",
#})

# emotion_query("Wie gehts?")

In [ ]:
"""
llm = OpenAI()
template = """Q: {question} A:"""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
"""

In [ ]:
"""
# from langchain.llms import OpenAI
from langchain.chains import LLMChain, ConstitutionalChain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple

qa_prompt = PromptTemplate(
    template="Q: {question} A:",
    input_variables=["question"],
)
qa_chain = LLMChain(llm=llm, prompt=qa_prompt)

constitutional_chain = ConstitutionalChain.from_llm(
    llm=None,
    chain=qa_chain,
    constitutional_principles=[
        ConstitutionalPrinciple(
            critique_request="Tell if this answer is good.",
            revision_request="Give a better answer.",
        )
    ],
)

constitutional_chain.run(question="What is the meaning of life?")
"""

# Notes

1. Langchain does not have support for classification models: https://python.langchain.com/en/latest/modules/models.html

    - Will have to figure out out-of-band retrieval and execution of feedback models that are not LLM's.

2. Can add steps to chain to capture text at various points in a chain: https://python.langchain.com/en/latest/reference/modules/chains.html#langchain.chains.SequentialChain .


# Links

- https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/llama#transformers.LlamaForCausalLM

- https://huggingface.co/docs/transformers/main_classes/text_generation


# Pinecone



In [ ]:
from dataclasses import dataclass
from typing import Any, Dict
from langchain.input import get_color_mapping
import pandas as pd

"""
@dataclass
class TruFeedback():
    typ: str
    input: LLMChain = None
    output: LLMChain = None

    def similar_to(aval: 'TruSentiment'):
        ...

@dataclass
class TruCheck():
    chain: LLMChain
    feedback: TruFeedback

    test: Callable[[Any]]

    pass
"""

# TruMetric(
#   TruFeedback("sentiment", output=llm_chain)
# )
# TruTest(
#   TruFeedback("sentiment", input=llm_chain).similar_to(TruFeedback("sentiment", output=llm_chain))
# )

class TruFeedbackChain(SimpleSequentialChain):

    feedback_pipeline: Any = None
    feedback_name: str = None
    feedback_records: pd.DataFrame = None

    def __init__(self, feedback="sentiment", *args, **kwargs):
        SimpleSequentialChain.__init__(self, *args, **kwargs)

        # others:
        # - relevance
        # - 
        if feedback == "sentiment":
            self.feedback_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
            self.feedback_name = "sentiment"
        elif feedback == "language":
            self.feedback_pipeline = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")
            self.feedback_name = "language"
        else:
            raise ValueError(f"Unsupported feedback type {feedback}")

        chains = kwargs['chains']
        record_cols = dict()
        for i, k in enumerate(chains):
            record_cols[f"{i}: {k.__class__.__name__} input"] = []
            record_cols[f"{i}: {k.__class__.__name__} output"] = []
            record_cols[f"{i}: {k.__class__.__name__} input {self.feedback_name}"] = []
            record_cols[f"{i}: {k.__class__.__name__} output {self.feedback_name}"] = []

        self.feedback_records = pd.DataFrame(record_cols)

        assert self.feedback_pipeline is not None

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        _input = inputs[self.input_key]
        color_mapping = get_color_mapping([str(i) for i in range(len(self.chains))])

        row = dict()

        for i, chain in enumerate(self.chains):
            print(f"{i} input:", _input)
            feedback = self.feedback_pipeline(_input)

            row[f"{i}: {chain.__class__.__name__} input"] = _input
            row[f"{i}: {chain.__class__.__name__} input {self.feedback_name}"] = feedback[0]['label']

            _input = chain.run(_input)

            print(f"{i} output:", _input, self.feedback_pipeline(_input))
            feedback = self.feedback_pipeline(_input)
            row[f"{i}: {chain.__class__.__name__} output"] = _input
            row[f"{i}: {chain.__class__.__name__} output {self.feedback_name}"] = feedback[0]['label']

            if self.strip_outputs:
                _input = _input.strip()
            self.callback_manager.on_text(
                _input, color=color_mapping[str(i)], end="\n", verbose=self.verbose
            )

        self.feedback_records.loc[len(self.feedback_records)] = row

        return {self.output_key: _input}

In [ ]:
feedback_chain = TruFeedbackChain(
    feedback="language",
    chains=[llm_chain, llm_chain_2],
    input_key="question"
)

In [ ]:
feedback_chain.run(question="Wie gehts?")
feedback_chain.run(question="How are you?")

In [ ]:
feedback_chain.feedback_records

In [ ]:
# ! pip install accelerate transformers bitsandbytes
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch
model = T5ForConditionalGeneration.from_pretrained("google/flan-ul2", device_map="auto", load_in_8bit=True)#, torch_dtype=torch.int8)
tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")

input_string = "Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?"

inputs = tokenizer(input_string, return_tensors="pt").input_ids.to("cuda:0")
outputs = model.generate(inputs, max_length=200)

print(tokenizer.decode(outputs[0]))
# <pad> They have 23 - 20 = 3 apples left. They have 3 + 6 = 9 apples. Therefore, the answer is 9.</s>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xxl")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xxl", torch_dtype=torch.bfloat16)

In [ ]:
pipe = pipeline(
    "conversational",
    #"conditional-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=16,
    device_map="auto",
    early_stopping=True
    # generate_kwargs=dict()
 #   device="auto"
)
# llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from transformers import Conversation

pipe(conversations=[Conversation("hello there")])

In [ ]:
chain

In [ ]:
dir(chain)

In [ ]:
from slackbot import chain
import langchain
import dill

In [ ]:
# from llama.hf import LLaMATokenizer

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import LLMChain, PromptTemplate
import torch

model_id = "gpt2"
# model_id = "decapoda-research/llama-7b-hf"
#model_id = "decapoda-research/llama-13b-hf"

model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.float16, local_files_only=True)#.to("cuda:0")
print("model loaded")
tokenizer = AutoTokenizer.from_pretrained(model_id, local_files_only=True)
print("tokenizer loaded")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=16,
    device_map="auto",
    early_stopping=True
    # generate_kwargs=dict()
 #   device="auto"
)
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from tru import TruChain

# llm = OpenAI()
template = """Q: {question} A:"""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

tru_chain = TruChain(chain=llm_chain)

tru_chain.run(dict(question="How are you?"))

In [ ]:
tru_chain.run(dict(question="How are you today?"))

In [ ]:
tru_chain.records

In [ ]:
tru_chain.model

In [ ]:
from langchain.chains import (ConversationalRetrievalChain,
                              SimpleSequentialChain)

verb = False

template = """Q: {question} A:"""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=verb)

template_2 = """Reverse this sentence: {sentence}."""
prompt_2 = PromptTemplate(template=template_2, input_variables=["sentence"])
llm_chain_2 = LLMChain(prompt=prompt_2, llm=llm, verbose=verb)

# print(llm_chain.run(question="What is the average air speed velocity of a laden swallow?"))

print(llm_chain_2.run(sentence="How are you doing?"))

seq_chain = SimpleSequentialChain(chains=[llm_chain, llm_chain_2], input_key="question", output_key="answer")
seq_chain.run(question="What is the average air speed velocity of a laden swallow?")

In [ ]:
tru_chain_2 = TruChain(seq_chain)

In [ ]:
seq_chain.run(question="What is the average air speed velocity of a laden swallow? again")
tru_chain_2.run(question="What is the average air speed velocity of a laden swallow?")

In [ ]:
tru_chain_2.records

In [ ]:
tru_chain_2.run(question="What is the average air speed velocity of a laden swallow?")

In [ ]:
tru_chain_2.model